In [8]:
import sqlite3
import pandas as pd

In [9]:
query = """
SELECT * FROM sensor_table

"""

with sqlite3.connect('../datasets/dallas.sqlite') as db:
    sensor_data = pd.read_sql(query, db)
    
sensor_data

,sensor_index,name,latitude,longitude
0,2644,Sachse Farms,32.990840,-96.599300
1,9504,Rosebud,32.419700,-97.008156
2,12969,Meadow Glen,32.991665,-96.859150
3,13013,GPTX,32.766370,-97.037926
4,16271,Arbormont Estates,32.882004,-97.084130
...,...,...,...,...
65,182041,CleanAIRE NC Hickory Creek,33.126100,-97.059390
66,184053,Creekwood Estates,32.909042,-97.124160
67,196323,Duck Creek Air Quality,32.960114,-96.692200
68,196421,Kilmichael Lane - PAII,32.977444,-96.776110


Problem : How to use python notebook to call bigquery?

In [2]:
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client()

# Write your SQL query
query = """
with my_location as (
  SELECT  ST_GEOGPOINT(-83.94, 35.97) as my_location,
          'Knoxville' as home
), 

stations as (
  SELECT *, ST_GEOGPOINT(lon,lat) as latlon_geo
  FROM `bigquery-public-data.noaa_gsod.stations` 
), 

get_closest as (
  SELECT home,my_location, st.*, 
  FROM (
    SELECT 
           ST_ASTEXT(my_location) as my_location, 
           home,
           ARRAY_AGG( # get the closest station
              STRUCT(usaf,wban,name,lon,lat,country,state,
                    ST_DISTANCE(my_location, b.latlon_geo)*0.00062137 as miles)
           ) as stations
    FROM my_location a, stations b
    WHERE ST_DWITHIN(my_location, b.latlon_geo, 32187)  --meters = 20 miles
    GROUP BY my_location, home
  ), UNNEST(stations) as st
  WHERE st.miles = (  -- Filter for station with minimum distance
    SELECT MIN(miles) FROM UNNEST(stations)
  )
)
SELECT  gc.*, 
    gs.temp AS temperature,
    gs.wdsp AS wind_speed,
    gs.gust AS wind_gust,
    gs.mxpsd AS max_wind_speed,
    gs.prcp AS precipitation
FROM get_closest gc, `bigquery-public-data.noaa_gsod.gsod20*` gs
WHERE max != 9999.9  -- code for missing data
AND   DATE(gs.date) BETWEEN '2023-01-01' AND '2023-01-31'  -- Date range for 2023
AND   gc.usaf = gs.stn  
AND   gc.wban = gs.wban
ORDER BY miles ASC;
"""

# Execute the query
query_job = client.query(query)

# Fetch results into a DataFrame
result_df = query_job.to_dataframe()

# Display the DataFrame
print(result_df)


         home          my_location    usaf   wban                       name  \
0   Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
1   Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
2   Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
3   Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
4   Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
5   Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
6   Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
7   Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
8   Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
9   Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
10  Knoxville  POINT(-83.94 35.97)  723264  99999  KNOXVILLE DOWNTOWN ISLAND   
11  Knoxville  POINT(-83.94 35.97)  7232

In [3]:
result_df

,home,my_location,usaf,wban,name,lon,lat,country,state,miles,temperature,wind_speed,wind_gust,max_wind_speed,precipitation
0,Knoxville,POINT(-83.94 35.97),723264,99999,KNOXVILLE DOWNTOWN ISLAND,-83.874,35.964,US,TN,3.713998,45.9,5.4,15.9,11.1,0.0
1,Knoxville,POINT(-83.94 35.97),723264,99999,KNOXVILLE DOWNTOWN ISLAND,-83.874,35.964,US,TN,3.713998,46.8,4.2,26.0,17.1,0.0
2,Knoxville,POINT(-83.94 35.97),723264,99999,KNOXVILLE DOWNTOWN ISLAND,-83.874,35.964,US,TN,3.713998,40.6,1.2,11.1,8.9,0.0
3,Knoxville,POINT(-83.94 35.97),723264,99999,KNOXVILLE DOWNTOWN ISLAND,-83.874,35.964,US,TN,3.713998,51.1,2.3,35.0,19.0,0.0
4,Knoxville,POINT(-83.94 35.97),723264,99999,KNOXVILLE DOWNTOWN ISLAND,-83.874,35.964,US,TN,3.713998,44.0,0.1,999.9,2.9,0.0
5,Knoxville,POINT(-83.94 35.97),723264,99999,KNOXVILLE DOWNTOWN ISLAND,-83.874,35.964,US,TN,3.713998,42.7,3.3,15.9,11.1,0.0
6,Knoxville,POINT(-83.94 35.97),723264,99999,KNOXVILLE DOWNTOWN ISLAND,-83.874,35.964,US,TN,3.713998,37.5,1.0,999.9,6.0,0.0
7,Knoxville,POINT(-83.94 35.97),723264,99999,KNOXVILLE DOWNTOWN ISLAND,-83.874,35.964,US,TN,3.713998,46.3,0.2,999.9,4.1,0.0
8,Knoxville,POINT(-83.94 35.97),723264,99999,KNOXVILLE DOWNTOWN ISLAND,-83.874,35.964,US,TN,3.713998,41.9,0.6,999.9,5.1,0.0
9,Knoxville,POINT(-83.94 35.97),723264,99999,KNOXVILLE DOWNTOWN ISLAND,-83.874,35.964,US,TN,3.713998,48.0,4.4,22.9,17.1,0.0


In [17]:
# build a function to fetch weather data 

def fetch_weather_data(latitude, longitude, date_range):
    # Define the SQL query to fetch weather data for the given location and date range
    
    query = f"""
    with sensor_location as (
    SELECT  ST_GEOGPOINT({longitude}, {latitude}) as sensor_location
    ), 

    stations as (
    SELECT *, ST_GEOGPOINT(lon,lat) as latlon_geo
    FROM `bigquery-public-data.noaa_gsod.stations` 
    ), 

    get_closest as (
    SELECT sensor_location, st.*, 
    FROM (
        SELECT 
            ST_ASTEXT(sensor_location) as sensor_location,
            ARRAY_AGG( # get the closest station
                STRUCT(usaf,wban,name,lon,lat,country,state,
                        ST_DISTANCE(sensor_location, b.latlon_geo)*0.00062137 as miles)
            ) as stations
        FROM sensor_location a, stations b
        WHERE ST_DWITHIN(sensor_location, b.latlon_geo, 32187)  --meters = 20 miles
        GROUP BY sensor_location
    ), UNNEST(stations) as st
    WHERE st.miles = (  -- Filter for station with minimum distance
        SELECT MIN(miles) FROM UNNEST(stations)
    )
    )
    SELECT  gc.*, 
        gs.temp AS temperature,
        gs.wdsp AS wind_speed,
        gs.gust AS wind_gust,
        gs.mxpsd AS max_wind_speed,
        gs.prcp AS precipitation
    FROM get_closest gc, `bigquery-public-data.noaa_gsod.gsod20*` gs
    WHERE max != 9999.9  -- code for missing data
    AND   DATE(gs.date) BETWEEN '{date_range[0]}' AND '{date_range[1]}'
    AND   gc.usaf = gs.stn  
    AND   gc.wban = gs.wban
    ORDER BY miles ASC;
    """

    # Execute the query
    query_job = client.query(query)

    # Fetch results into a DataFrame
    result_df = query_job.to_dataframe()

    return result_df

In [15]:
date_range = ('2023-01-01', '2023-01-03')

In [18]:
for index, row in sensor_data.iterrows():
    
    weather_data = fetch_weather_data(row['latitude'], row['longitude'], date_range)
    print(weather_data)

            sensor_location    usaf   wban                           name  \
0  POINT(-96.5993 32.99084)  721044  00448  RALPH M HALL ROCKWALL MUNICIP   
1  POINT(-96.5993 32.99084)  721044  00448  RALPH M HALL ROCKWALL MUNICIP   
2  POINT(-96.5993 32.99084)  721044  00448  RALPH M HALL ROCKWALL MUNICIP   

      lon     lat country state      miles  temperature wind_speed  wind_gust  \
0 -96.435  32.931      US    TX  10.383485         64.6        7.0       17.1   
1 -96.435  32.931      US    TX  10.383485         68.1        8.7       27.0   
2 -96.435  32.931      US    TX  10.383485         62.4       10.9       19.0   

  max_wind_speed  precipitation  
0           12.0            0.0  
1           19.0            0.0  
2           15.9            0.0  
Empty DataFrame
Columns: [sensor_location, usaf, wban, name, lon, lat, country, state, miles, temperature, wind_speed, wind_gust, max_wind_speed, precipitation]
Index: []
              sensor_location    usaf   wban             na